Install all the required libraries for implementing retrieval-augmented generation (RAG), large language model (LLM) integrations, embedding-based similarity search, and dataset handling, ensuring a unified environment for counter-narrative generation experiments.

In [ ]:
!pip install -U langgraph langchain_openai google-generativeai langchain-community langchain-google-genai langchain-anthropic anthropic sentence-transformers faiss-cpu datasets

Import all the required API token keys for all the models used


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = '<OPENAI_API_KEY>'
os.environ['HF_TOKEN'] = '<HF_TOKEN>'
os.environ['GOOGLE_API_KEY'] = '<GOOGLE_API_KEY>'
os.environ['ANTHROPIC_API_KEY'] = '<ANTHROPIC_API_KEY>'

In [ ]:
from typing import Dict, TypedDict

class GraphState(TypedDict):
  """
  Represents the state of an agent in the conversation.
  """

  keys: Dict[str, any]

First step is to generate contextual queries based on the Hate speech

In [ ]:
import re
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

def generate_queries(state):
  from langchain_core.documents import Document

  state_dict = state["keys"]
  hate_speech = state_dict["hate_speech"]

  template = """
  பின்வரும் தமிழில் உள்ள வெறுப்புப் பேச்சு கூறுக்கு எதிராக ஒரு தரமான எதிர் பேச்சை உருவாக்க வேண்டியுள்ளது:

"{hatespeech}"

இந்தக் குறிப்பிட்ட வெறுப்புப் பேச்சுக்கு பொருந்தக்கூடிய எதிர் பேச்சை உருவாக்க பயன்படுத்தத்தக்க 3 ஆய்வுக் கேள்விகளை உருவாக்கவும்.

- கேள்விகள் தவிர வேறு உரை (முன்னுரை, விளக்கம்) இட வேண்டாம்.
- ஒவ்வொரு கேள்வியும் தனித்தனி வரியில் இடம்பெற வேண்டும்.
- அனைத்து கேள்விகளும் தமிழில் மட்டும் இருக்க வேண்டும்.
- ஒவ்வொரு கேள்வியும் 1., 2., 3. என்ற எண்ணுக்குறிப்புடன் தொடங்க வேண்டும்.
  """

  query_template = ChatPromptTemplate.from_template(template)

  chain = query_template | llm | StrOutputParser()

  output = chain.invoke({"hatespeech": hate_speech})
  queries = re.findall(r"\d+\.\s*(.*)", output.strip())

  return {"keys": {"hate_speech": hate_speech, "queries": queries}}

Loads the HinduTamil-News-Articles-Dataset to get the fact based counter narratives


In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

dataset = load_dataset("Shwetasss/HinduTamil-News-Articles-Dataset", split="train")
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

corpus = [doc['Text'] for doc in dataset]
corpus_embeddings = embedder.encode(corpus, convert_to_numpy=True, show_progress_bar=True)

dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)

id_map = {i: doc for i, doc in enumerate(dataset)}


*   Retrieves the top 2 relevant documents based on the hate speech and contextual queries
*   Summarizes the documents to 2 - 3 sentences
*   Filters relevant documents based on hate speech and the query
*   Generates three candidate counter narratives




In [ ]:
#from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_anthropic import ChatAnthropic

def retrieve_documents_tamil(state, top_k=2):
    state_dict = state["keys"]
    hate_speech = state_dict["hate_speech"]
    queries = state_dict["queries"]

    query_document_pairs = []
    query_embeddings = embedder.encode(queries, convert_to_numpy=True)

    for query, query_embedding in zip(queries, query_embeddings):
        distances, indices = index.search(np.array([query_embedding]), top_k)
        for idx in indices[0]:
            document = id_map[idx]
            #print(f"document is : {document}")
            query_document_pairs.append({
                "query": query,
                "document": {
                    "title": document['Text']
                }
            })
    #print(query_document_pairs)
    return {"keys": {"hate_speech": hate_speech, "queries": queries, "query_document_pairs": query_document_pairs}}

def summarize_documents(state):
  state_dict = state["keys"]
  hate_speech = state_dict["hate_speech"]
  queries = state_dict["queries"]
  query_document_pairs = state_dict["query_document_pairs"]

  summary_prompt = """
    இந்த ஆவணத்தை 2 அல்லது 3 வசனங்களில் சுருக்கவும். சுருக்கம், கீழ்க்கண்ட கேள்விக்கு பதிலளிக்க வேண்டும் மற்றும் வெறுப்புப் பேச்சுக்கு எதிரான தகவல்களை உள்ளடக்கியதாக இருக்க வேண்டும். தொடர்புடைய புள்ளிவிவரங்கள் மற்றும் தகவல் மூலங்களை விட்டுவிடாமல் சேர்க்கவும்.

    கேள்வி: {query}

    வெறுப்புப் பேச்சு: {hs}

    ஆவணம்: {document}

  """
  prompt = ChatPromptTemplate.from_template(summary_prompt)

  chain = prompt | llm | StrOutputParser()

  query_summarized_document_pairs = []

  for query_document_pair in query_document_pairs:
    document_summary = chain.invoke({"query":query_document_pair["query"],"document":query_document_pair["document"], "hs":hate_speech})
    query_summarized_document_pairs.append({"query": query_document_pair["query"], "document": document_summary})

  #print(f"query_summarized_document_pairs : {query_summarized_document_pairs}")

  return {"keys": {"hate_speech": hate_speech, "queries": queries, "query_document_pairs": query_document_pairs, "query_summarized_document_pairs": query_summarized_document_pairs}}

def grade_documents(state):
  from langchain_core.documents import Document
  state_dict = state["keys"]
  hate_speech = state_dict["hate_speech"]
  queries = state_dict["queries"]
  query_document_pairs = state_dict["query_document_pairs"]
  query_summarized_document_pairs = state_dict["query_summarized_document_pairs"]

  #print(f"Summarized document pairs:  {query_summarized_document_pairs}")

  grade_prompt = """
  நீங்கள் வெறுப்புப் பேச்சுக்கு எதிராக ஒரு எதிர்பேச்சு உருவாக்க பயன்படும் பதிவுக்கான தொடர்பை மதிப்பீடு செய்யும் ஒரு மதிப்பீட்டாளர்.

  பின்வரும் தகவல்களை கவனமாக படிக்கவும்:

  - வெறுப்புப் பேச்சு: {hs}

  - தேடலுக்காக உருவாக்கப்பட்ட கேள்வி: {question}

  - தேடப்பட்ட ஆவணம்: {document}

  இந்த ஆவணம், மேலே கொடுக்கப்பட்ட வெறுப்புப் பேச்சுக்கு எதிராக ஒரு அர்த்தமுள்ள எதிர்பேச்சு உருவாக்குவதற்கு பயன்படுமா என்பதை மதிப்பீடு செய்ய வேண்டும்.

  **தரவரிசை அளவுகள்:**

  - **1** — ஆவணத்தில் உள்ள தகவல்கள், எதிர்பேச்சு உருவாக்குவதற்கு உதவக்கூடியவையாக இருந்தால்.

  - **0** — ஆவணம் வெறுப்புப் பேச்சுடன் ஏற்கெனவே ஒத்துப் போனால் அல்லது எதிர்பேச்சு உருவாக்குவதற்கு பயன்படாததெனத் தோன்றினால்.

  உங்கள் பதிலை எந்தவித விளக்கமும் இல்லாமல், வெறும் ஒரு இலக்கமாக (0 அல்லது 1) மட்டும் பதியவும்.

  """

  prompt_template = ChatPromptTemplate.from_template(grade_prompt)

  filtered_query_document_pairs = []

  for query_summarized_document_pair in query_summarized_document_pairs:
      query = query_summarized_document_pair["query"]
      document = query_summarized_document_pair["document"]

      chain = prompt_template | llm | StrOutputParser()
      score = chain.invoke({
          "document": document,
          "hs": hate_speech,
          "question": query
      }).strip()

      if score.lower() == "1":
          #print("Document is relevant!")
          filtered_query_document_pairs.append(query_summarized_document_pair)
      else:
          print("Document is irrelevant!")

  return {
        "keys": {
            "hate_speech": hate_speech,
            "queries": queries,
            "query_document_pairs": query_document_pairs,
            "query_summarized_document_pairs": query_summarized_document_pairs,
            "filtered_query_document_pairs": filtered_query_document_pairs
        }
    }


def generate_counterspeech(state):
  from langchain_core.documents import Document

  state_dict = state["keys"]
  hate_speech = state_dict["hate_speech"]
  queries = state_dict["queries"]
  query_document_pairs = state_dict["query_document_pairs"]
  query_summarized_document_pairs = state_dict["query_summarized_document_pairs"]
  filtered_query_document_pairs = state_dict["filtered_query_document_pairs"]

  cn_prompt = """
  பின்வரும் வெறுப்புப் பேச்சுக்கு எதிராக தரமான மூன்று எதிர் பேச்சுகளை (Counter Speech) உருவாக்கவும். கீழே வழங்கப்படும் தகவல்கள் பல்வேறு ஆதாரங்களிலிருந்து பெறப்பட்டுள்ளன. இத்தகவல்களை உங்கள் பதில்களில் உள்ளடக்கவும் மற்றும் தேவையான இடங்களில் மேற்கோள்கள் (sources) குறிப்பிடவும்.

  தகவல் சார்ந்த உள்ளடக்கம் : {context}

  வெறுப்புப் பேச்சு : {hatespeech}

  உங்கள் பதிலில், ஒவ்வொரு எதிர் பேச்சும் தனித்தனி வரியில் 1., 2., 3. என தொடங்க வேண்டும். ஒவ்வொன்றும் அதிகபட்சம் இரண்டு வரிகளாக மட்டுப்படுத்தப்பட வேண்டும்.

  """

  prompt = ChatPromptTemplate.from_template(cn_prompt)

  context = "\n\n".join([doc['document'] for doc in filtered_query_document_pairs])

  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
  #llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
  #llm = ChatAnthropic(model="claude-3-7-sonnet-20250219", temperature=0)

  chain = prompt | llm | StrOutputParser()
  gpt_counter_narratives = chain.invoke({"context":context,"hatespeech":hate_speech})
  #print(f"Counter Narratives are : {gpt_counter_narratives}")

  return {"keys":
            {"hate_speech": hate_speech,
            "queries": queries,
            "query_document_pairs": query_document_pairs,
            "filtered_query_document_pairs": filtered_query_document_pairs,
            "query_summarized_document_pairs": query_summarized_document_pairs,
            "gpt_4o_counterspeech": gpt_counter_narratives,
            }}



Generates the best Counter narrative based on the above three candidate counter narratives and uses the HITL approach V5 dataset to get the relevant examples for a given hate speech


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

def retrieve_top_k_examples(input_text, ref_texts, ref_cns, k=5):
    """
    Retrieves top-k most similar hate speech examples using cosine similarity.
    """
    query_embedding = model.encode([input_text], convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, ref_texts)[0]
    top_indices = similarities.topk(k=k).indices.tolist()

    top_examples = [(ref_texts[i], ref_cns[i]) for i in top_indices]
    return top_examples

def generate_best_counterspeech(state):
  from langchain_core.documents import Document

  ### HITL V5 as KB
  ref_df = pd.read_excel('tamil_hs_cn_v5.xlsx')
  test_df = pd.read_excel('golden_hs_cn_test_set.xlsx')

  ref_hs_list = ref_df['Hate speech'].fillna('').tolist()
  ref_cn_list = ref_df['Counter Narrative from dataset'].fillna('').tolist()
  ref_embeddings = model.encode(ref_hs_list, convert_to_tensor=True)


  state_dict = state["keys"]
  hate_speech = state_dict["hate_speech"]
  queries = state_dict["queries"]
  query_document_pairs = state_dict["query_document_pairs"]
  query_summarized_document_pairs = state_dict["query_summarized_document_pairs"]
  filtered_query_document_pairs = state_dict["filtered_query_document_pairs"]
  counter_narratives = state_dict["gpt_4o_counterspeech"]

  cn_prompt = """
  பின்வரும் வெறுப்புப் பேச்சுக்கு எதிராக, தகவல் மற்றும் உண்மைகளில் அடிப்படையிலான, ஒரு தரமான, தெளிவான மற்றும் சமூக பொறுப்புடன் கூடிய எதிர்பேச்சை உருவாக்கவும்.

  கீழே வழங்கப்பட்டுள்ள மூன்று எதிர்பேச்சுகளும் தத்தமதாக சில சிறப்பம்சங்களை கொண்டுள்ளன. அவற்றில் சிறந்த கூறுகளை எடுத்துக்கொண்டு, ஒரு சுருக்கமான (அதிகபட்சம் 2 வரிகள்), உண்மை சார்ந்த மற்றும் உறுதியான எதிர்பேச்சை உருவாக்கவும். தேவையான இடங்களில் ஆதாரங்களை (sources) குறிப்பிடவும்.

- வெறுப்புப் பேச்சு:   {hatespeech}

- தகவல் சார்ந்த உள்ளடக்கம்:   {context}

- மூன்று எதிர்பேச்சுகள்:   {counter_narratives}

- உதாரணங்கள்:
  """

  top_k = retrieve_top_k_examples(hate_speech, ref_hs_list, ref_cn_list, k=5)
  #print(f"Examples of hs {hate_speech} are {top_k}")

  for hs, cn in top_k:
      cn_prompt += f"வெறுப்புப் பேச்சு: {hs}\nஎதிர்வினை: {cn}\n\n"

  prompt = ChatPromptTemplate.from_template(cn_prompt)

  context = "\n\n".join([doc['document'] for doc in filtered_query_document_pairs])

  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
  #llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
  #llm = ChatAnthropic(model="claude-3-7-sonnet-20250219", temperature=0)

  chain = prompt | llm | StrOutputParser()
  counter_narrative = chain.invoke({"context":context,"hatespeech":hate_speech, "counter_narratives": counter_narratives})
  print(f"Final Counter Narrative is : {counter_narrative}")

  return {"keys":
            {"hate_speech": hate_speech,
            "queries": queries,
            "query_document_pairs": query_document_pairs,
            "filtered_query_document_pairs": filtered_query_document_pairs,
            "query_summarized_document_pairs": query_summarized_document_pairs,
            "gpt_4o_counterspeech": counter_narratives,
            "final_counter_narrative": counter_narrative,
            }}

Defines and compiles a state-based counter-narrative generation pipeline using LangGraph. The pipeline is structured as a directed graph where each node represents a modular task in the generation process. The flow begins with query generation, followed by document retrieval in Tamil, document summarization, and document grading to filter the most relevant evidence. The graded content is then used to generate multiple counter-speech candidates, from which the best counter-speech is selected.




In [ ]:
from langgraph.graph import END, StateGraph
from langchain_core.documents import Document

pipeline = StateGraph(GraphState)

pipeline.add_node("generate_queries", generate_queries)
pipeline.add_node("retrieve_documents_tamil", retrieve_documents_tamil)
pipeline.add_node("summarize_documents", summarize_documents)
pipeline.add_node("grade_documents", grade_documents)
pipeline.add_node("generate_counterspeech", generate_counterspeech)
pipeline.add_node("generate_best_counterspeech", generate_best_counterspeech)

pipeline.set_entry_point("generate_queries")
pipeline.add_edge("generate_queries", "retrieve_documents_tamil")
pipeline.add_edge("retrieve_documents_tamil", "summarize_documents")
pipeline.add_edge("summarize_documents", "grade_documents")
pipeline.add_edge("grade_documents", "generate_counterspeech")
pipeline.add_edge("generate_counterspeech", "generate_best_counterspeech")

app = pipeline.compile()

This code applies the counter-narrative generation pipeline to each hate speech instance in golden_hs_cn_test_set.xlsx. For every input, it extracts the best fact-based counter-narrative along with the supporting factual context from retrieved documents. The results are appended to a new dataset.


In [ ]:
import pandas as pd

df = pd.read_excel("golden_hs_cn_test_set.xlsx")

ta_counter_narratives = []
ta_context = []

for hs_text in df['Hate speech']:
  inputs = {"keys": {"hate_speech": hs_text}}

  tamil_cn = ""

  for output in app.stream(inputs):
    for key, value in output.items():

      if key == "generate_best_counterspeech":
        results = value["keys"]
        print(f"results : {results}")
        tamil_cn = results.get("final_counter_narrative")
        print(f"tamil_cn : {tamil_cn}")
        context = results.get("filtered_query_document_pairs")

  ta_counter_narratives.append(tamil_cn)
  ta_context.append(context)


df["fact_counter_narratives"] = ta_counter_narratives
df["fact_context"] = ta_context

df.to_excel("fact_counter_narratives.xlsx", index=False)

**Zero-Shot Prompting**

This approach uses the GPT-4o model to generate zero-shot counter-narratives for each hate speech sample in golden_hs_cn_test_set.xlsx. A structured Tamil prompt guides the model to produce short, respectful, and fact-based responses tailored to the hate speech category. The generated counter-narratives are stored in a new column and exported as zero_shot_cn_gpt4o.xlsx for further evaluation.

In [ ]:
import pandas as pd
import openai
from openai import OpenAI
import time
from tqdm import tqdm

client = openai.OpenAI(api_key="")

df = pd.read_excel("golden_hs_cn_test_set.xlsx")
df = df.dropna(subset=['Hate speech'])

df["Generated_CN_GPT4o"] = ""

def build_prompt(hs_text: str, category: str) -> str:
    return f"""
நீங்கள் ஒரு சமூக பொறுப்புள்ள மொழி உருவாக்கும் நுண்ணறிவு மாடலாக செயல்படுகிறீர்கள். கீழே கொடுக்கப்பட்டுள்ள '{category}' வகையைச் சேர்ந்த வெறுப்புப் பேச்சுக்கு (Hate Speech) ஒரு உண்மையியல் அடிப்படையிலான, மதிப்புமிக்க எதிர்பேச்சை (Counter Narrative) உருவாக்கவும்.

தயவுசெய்து பின்வரும் வழிகாட்டுதல்களை பின்பற்றவும்:
- வெறுப்பை வன்முறையற்ற முறையில் எதிர்த்து, சமூக ஒற்றுமையை வலியுறுத்த வேண்டும்.
- உண்மைத் தரவுகள் அல்லது பொதுத் தகவல்களைக் கொண்டு ஆதரிக்கப்பட வேண்டும்.
- அதிகபட்சம் இரண்டு வரிகளுக்குள் சுருக்கமாக இருக்க வேண்டும்.
- யாரையும் நையப்புடை செய்யாமல், மதிப்போடு பதிலளிக்க வேண்டும்.

வெறுப்புப் பேச்சு:
\"{hs_text}\"

உங்கள் எதிர்பேச்சு:
"""

# Loop through HS samples and generate CNs
for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        prompt = build_prompt(row['Hate speech'], row['Category'])

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "நீங்கள் சமூக பொறுப்புடன் செயல்படும் உதவியாளராகும். வெறுப்புப் பேச்சுக்கு எதிராக உண்மைத் தரவுகளின் அடிப்படையில் மரியாதைமிக்க, உணர்வுபூர்வமான மற்றும் தகவலளிக்கும் பதில்களை, அதிகபட்சம் இரண்டு வரிகளில் உருவாக்க பயிற்சி பெற்றுள்ளீர்கள்."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150,
        )

        generated_cn = response.choices[0].message.content.strip()
        df.at[idx, "Zero_Shot_CN_GPT4o"] = generated_cn

        time.sleep(1)

    except Exception as e:
        print(f"Error at index {idx}: {e}")
        df.at[idx, "Zero_Shot_CN_GPT4o"] = "ERROR"

df.to_excel("zero_shot_cn_gpt4o.xlsx", index=False)


**Few-Shot Prompting**

This approach generates few-shot counter-narratives for Tamil hate speech using GPT-4o, guided by a prompt that includes 10 example hate speech counter narrative pairs as demonstrations. The generated outputs are stored in a new column and saved to few_shot_10_cn_gpt4o.xlsx for further evaluation.


In [ ]:
import pandas as pd
import openai
from openai import OpenAI
import time
from tqdm import tqdm

client = openai.OpenAI(api_key="")

# Load your dataset
df = pd.read_excel("golden_hs_cn_test_set.xlsx")
df = df.dropna(subset=['Hate speech'])

df["Generated_CN_GPT4o"] = ""

def build_prompt(hs_text: str, category: str) -> str:
    return f"""
நீங்கள் ஒரு சமூக பொறுப்புடன் செயல்படும் உதவியாளர். கீழே சில வெறுப்புப் பேச்சுகள் மற்றும் அவற்றுக்கான உண்மை மற்றும் மரியாதையுடன் அமைந்த எதிர்ப்புப் பேச்சுகள் (Counter Narratives) வழங்கப்பட்டுள்ளன. இந்த எடுத்துக்காட்டுகளின் அடிப்படையில், பின்வரும் புதிய வெறுப்புப் பேச்சுக்கு ஒரு சுருக்கமான (அதிகபட்சம் 2 வரிகள்), உண்மையைக் கொண்ட மற்றும் சமூக ஒற்றுமையை ஊக்குவிக்கும் எதிர் பேச்சை உருவாக்கவும்.

உதாரணங்கள்:

1.
வெறுப்புப் பேச்சு: இது எங்க நாடு அனைவருக்கும் பிரித்தானியர் நாடு தான் பெண்களும் குடியேறிய சொல்லிகளும் பலம் பெற்றுவருவார்கள்
எதிர் பேச்சு: அனைவரும் ஒன்றாக வாழும் இந்த நாட்டில் மதம், மொழி, வரலாறு எதுவாக இருந்தாலும், அனைவரும் சமபாதமாக இருக்க வேண்டும்.

2.
வெறுப்புப் பேச்சு: வெளிநாட்டு முஸ்லிம்கள் இங்கே வந்து எங்களுக்கு பயம் கொடுக்கிறார்கள், தலைமுறை வேண்டாம் சாதிக்கலாம்
எதிர் பேச்சு: எல்லா சமூகங்களும் இங்கே சமமாக வாழக்கூடிய உரிமையுடையவர்கள்; பயத்தையும் வெறுப்பையும் இல்லாமல் ஒருமைப்பாட்டை வளர்க்க வேண்டியது நம் கடமை.

3.
வெறுப்புப் பேச்சு: அதே போல் பெண்களும்...... ஆண்கள் அதிகம் சம்பளம் வாங்க வேண்டும் என்ற எண்ணத்தை....கை விட வேண்டும்
எதிர் பேச்சு: வேலைக்கான சம்பள வேறுபாடு சமூகத்தில் ஆழமான பாகுபாடை காட்டுகிறது. சமபள உரிமை அனைவருக்கும் இருக்க வேண்டும்.

4.
வெறுப்புப் பேச்சு: ஆண்டாள் கைவிட்டாளே ராஜா
எதிர் பேச்சு: ஆண்டாள் ஒரு பக்தி சின்னம். அவளது பெயரை இழிவுபடுத்துவது பக்தியையும் பெண்மையையும் அவமதிப்பதாகும்.

5.
வெறுப்புப் பேச்சு: ஆண்டாள் விபச்சாரி தான் அதில் உண்மை உள்ளது
எதிர் பேச்சு: ஆண்டாள் பக்தி, ஈர்ப்பு, கவிதை – இவை அனைத்தையும் கொண்ட ஒரு பெண். அவளுக்கு விபச்சாரி என்று சொல்லுவது புனிதத்தையும் கலாச்சாரத்தையும் மாசுபடுத்துகிறது.

6.
வெறுப்புப் பேச்சு: இந்து மதத்தை வைத்து வாழும் ஈனபிறவி
எதிர் பேச்சு: எதுக்கு ஒரு மதத்தை இழிவாக பேசவேண்டும். தமிழ் நாட்டில் உள்ளவர்கள் திருந்துகள் அறிவு உள்ளவர்கள் இதை முதலில் யோசிங்கள்.

7.
வெறுப்புப் பேச்சு: யன்டா நா யே வட நாட்லெஇருந்து வந்தஉனகே இவ்வளவு வாய்னா  வைர முத்துஅவர்கள்  தமிழன் டா
எதிர் பேச்சு: சிலர் சாதியை சொல்லி; பிழைப்பு நடத்துவான்; சிலர் மதகலவரத்தை தூண்டிவிட்டு; பிழைப்பு நடத்துவான்; தமிழன் சாதியால் பிளவு பட்டு; இருக்கும் வரை இது போன்ற; மணிதர்களுக்கு கொண்டாட்டம் தான்

8.
வெறுப்புப் பேச்சு: வந்தேறி நாய்கள் ஓடுங்கடா உங்க நாட்டுக்கு எங்க வந்து என்ன ஆதிக்கம் பண்ணுரீங்க செருப்பு
எதிர் பேச்சு: இந்த மண்ணில் வாழும் ஒவ்வொருவருக்கும் சம உரிமை உண்டு.  பிறப்பால் யாரும் உயர்ந்தவரோ தாழ்ந்தவரோ இல்லை.  ஒற்றுமையாக வாழ்வதே நமது பலம்.

9.
வெறுப்புப் பேச்சு: படுக்கை விரித்து பல சாம்ராஜயங்களை வீழ்த்திய ஆரிய தேவிடியா மகனே உன் வரலாறு எப்படி என்று தெரியும்டா
எதிர் பேச்சு: பழைய வரலாறுகளை பெண்களை இழிவுபடுத்த பயனப்படுத்த வேண்டாம். பெண்ணின் பங்கு அரசியல், சமூகம், கலாச்சாரம் என பலதிலும் பெரிது. அவளது விலாசத்தை விமர்சனம் செய்யாமல், அவளது பங்களிப்பை பாராட்டுவோம்.

10.
வெறுப்புப் பேச்சு: vijai kumar; உன் ஆத்தா ஊரை ஓல்த்த தேவிடியா
எதிர் பேச்சு: அம்மா என்பது ஒரு புனிதமான சொல். பெண்களை இழிவுபடுத்தும் வார்த்தைகள் உங்கள் வளர்ச்சியையும் மதிப்பையும் காட்டுகிறது. கலாச்சாரம் என்பது மரியாதையில்தான் இருக்கும்.

இப்போது, கீழ்காணும் வெறுப்புப் பேச்சுக்கு சமமான துல்லியமான எதிர் பேச்சை உருவாக்கவும்:

வெறுப்புப் பேச்சு: \"{hs_text}\"

எதிர் பேச்சு:

"""

# Loop through HS samples and generate CNs
for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        prompt = build_prompt(row['Hate speech'], row['Category'])

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "நீங்கள் சமூக பொறுப்புடன் செயல்படும் உதவியாளராகும். வெறுப்புப் பேச்சுக்கு எதிராக உண்மைத் தரவுகளின் அடிப்படையில் மரியாதைமிக்க, உணர்வுபூர்வமான மற்றும் தகவலளிக்கும் பதில்களை, அதிகபட்சம் இரண்டு வரிகளில் உருவாக்க பயிற்சி பெற்றுள்ளீர்கள்."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150,
        )

        generated_cn = response.choices[0].message.content.strip()
        df.at[idx, "Few_Shot_10_CN_GPT4o"] = generated_cn

        time.sleep(1)

    except Exception as e:
        print(f"Error at index {idx}: {e}")
        df.at[idx, "Few_Shot_10_CN_GPT4o"] = "ERROR"

df.to_excel("few_shot_10_cn_gpt4o.xlsx", index=False)

**Using HITL-approach V5 as KB**


This approach implements a knowledge-based few-shot counter-narrative generation pipeline using GPT-4o, where top 5 similar examples are retrieved from the human-in-the-loop curated V5 dataset. Sentence-transformer embeddings are used to identify the most relevant HS CN pairs. The final outputs are stored in HITL_V5_KB_CN_Generation.xlsx.

In [ ]:
import pandas as pd
import openai
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

client = openai.OpenAI(api_key="")

ref_df = pd.read_excel('tamil_hs_cn_v5.xlsx')
test_df = pd.read_excel('golden_hs_cn_test_set.xlsx')

ref_hs_list = ref_df['Hate speech'].fillna('').tolist()
ref_cn_list = ref_df['Counter Narrative from dataset'].fillna('').tolist()
ref_embeddings = model.encode(ref_hs_list, convert_to_tensor=True)


def build_prompt_few_shot(input_hs, top_examples):
    """
    Constructs a few-shot prompt in Tamil using top similar examples.
    """
    instructions = (
        """
        நீங்கள் ஒரு சமூக பொறுப்புடன் செயல்படும் உதவியாளர். கீழே சில வெறுப்புப் பேச்சுகள் மற்றும் அவற்றுக்கான உண்மை மற்றும் மரியாதையுடன் அமைந்த எதிர்ப்புப் பேச்சுகள் (Counter Narratives) வழங்கப்பட்டுள்ளன. இந்த எடுத்துக்காட்டுகளின் அடிப்படையில், பின்வரும் புதிய வெறுப்புப் பேச்சுக்கு ஒரு சுருக்கமான (அதிகபட்சம் 2 வரிகள்), உண்மையைக் கொண்ட மற்றும் சமூக ஒற்றுமையை ஊக்குவிக்கும் எதிர் பேச்சை உருவாக்கவும்.

        உதாரணங்கள்:
        """
    )

    for hs, cn in top_examples:
        instructions += f"வெறுப்புப் பேச்சு: {hs}\nஎதிர்வினை: {cn}\n\n"

    instructions += f"வெறுப்புப் பேச்சு: {input_hs}\nஎதிர்வினை:"
    return instructions

def retrieve_top_k_examples(input_text, ref_texts, ref_cns, k=5):
    """
    Retrieves top-k most similar hate speech examples using cosine similarity.
    """
    query_embedding = model.encode([input_text], convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, ref_embeddings)[0]
    top_indices = similarities.topk(k=k).indices.tolist()

    top_examples = [(ref_texts[i], ref_cns[i]) for i in top_indices]
    return top_examples

def generate_counter_narrative(prompt):
    """
    Calls OpenAI GPT-4o model to generate CN.
    """
    response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "நீங்கள் சமூக பொறுப்புடன் செயல்படும் உதவியாளராகும். வெறுப்புப் பேச்சுக்கு எதிராக உண்மைத் தரவுகளின் அடிப்படையில் மரியாதைமிக்க, உணர்வுபூர்வமான மற்றும் தகவலளிக்கும் பதில்களை, அதிகபட்சம் இரண்டு வரிகளில் உருவாக்க பயிற்சி பெற்றுள்ளீர்கள்."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150,
        )

    return response.choices[0].message.content.strip()

# Process each test input
results = []
for hs_input in test_df['Hate speech'].fillna(''):
    top_k = retrieve_top_k_examples(hs_input, ref_hs_list, ref_cn_list, k=5)
    prompt = build_prompt_few_shot(hs_input, top_k)
    generated_cn = generate_counter_narrative(prompt)
    results.append({'Hate speech': hs_input, 'Generated CN': generated_cn})

output_df = pd.DataFrame(results)
output_df.to_excel("HITL_V5_KB_CN_Generation.xlsx", index=False)

**References**

*   Wilk, B., Shomee, H.H., Maity, S.K. and Medya, S., 2025, April. Fact-based Counter Narrative Generation to Combat Hate Speech. In Proceedings of the ACM on Web Conference 2025 (pp. 3354-3365).
*   https://python.langchain.com/docs/integrations/providers/openai/
*   https://python.langchain.com/docs/integrations/providers/anthropic/
*   https://python.langchain.com/docs/integrations/chat/google_generative_ai/
